# Instalación de dependencias y Configuración de API Key

1. Ve a [https://aistudio.google.com/](https://aistudio.google.com/) y sigue las instrucciones para generar un API Key

2. Configura tu API key en Colab: en el menú de la izquierda, clicka en la llave > Claves de API de Gemini > Importar desde Google AI Studio

In [6]:
# Instalamos las dependencias necesarias
!pip install gradio google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

# Parte 1 - Genera menús y listas de la compra con Gemini

1. Genera un menú semanal de estilo mediterráneo en formato tabla (comida y cena)


2. Genera la lista de la compra en base al menú

In [2]:
from google.colab import userdata
from google import genai
from google.genai import types

client = genai.Client(
    api_key=userdata.get("GOOGLE_API_KEY"),
)

def generate_text(user_prompt):
  response = client.models.generate_content(
      model="gemini-2.0-flash",
      contents=types.Content(
          role="user",
          parts=[
              types.Part.from_text(text=user_prompt),
          ],
      ),
  )
  return response.text

In [4]:
template = "Genera un menú semanal de estilo mediterráneo en formato tabla (comida y cena)"
menu = generate_text(template)

print(menu)

¡Aquí tienes un menú semanal de estilo mediterráneo, en formato tabla, con ideas para comidas y cenas! Este menú se centra en ingredientes frescos, aceite de oliva, legumbres, pescado, frutas y verduras.

**MENÚ SEMANAL MEDITERRÁNEO**

| Día     | Comida                                                                                                | Cena                                                                                                  |
|---------|--------------------------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------|
| **Lunes** | Ensalada griega con pollo a la plancha (pepino, tomate, aceitunas, queso feta, cebolla roja)             | Lubina al horno con patatas asadas con hierbas y espárragos                                       |
| **Martes**| Sopa minestrone (con verduras, pasta y alubias) + pan integral                          

In [3]:
user_prompt = "Genera un menú semanal basado en una dieta mediterránea con comida y cena. Solo devuelve la tabla de menú."

menu_generado = generate_text(user_prompt)

In [4]:
user_prompt_template = "Genera la lista de la compra para este menú {menu_generado}. Devuelve solo la lista."

user_prompt = user_prompt_template.format(menu_generado=menu_generado)

print(generate_text(user_prompt))

**Verduras y Hortalizas:**

*   Tomates
*   Pepino
*   Cebolla roja
*   Calabacín
*   Berenjena
*   Pimiento (varios colores)
*   Patatas
*   Ajo
*   Brócoli
*   Espárragos trigueros
*   Calabaza
*   Cebolla
*  Pimientos (para brochetas y paella)

**Legumbres:**

*   Garbanzos (secos o en conserva)
*   Lentejas

**Pasta y Granos:**

*   Pasta integral (para ensalada)
*   Quinoa
*   Couscous
*   Arroz (para paella)

**Proteínas:**

*   Lubina
*   Atún (en lata)
*   Huevos
*   Pollo (pechugas)
*   Salmón
*   Cordero (para brochetas)
*   Conejo

**Lácteos:**

*   Mozzarella (para pizza)

**Otros:**

*   Aceitunas
*   Aceite de oliva virgen extra
*   Vinagre
*   Sal
*   Pimienta
*   Especias (ajo en polvo, pimentón dulce/picante - para el conejo y otros platos)
*   Tomate triturado (para la pizza y el gazpacho)
*   Masa para pizza integral (o ingredientes para hacerla)

**Opcional (según gusto):**

*   Hierbas frescas (perejil, albahaca, etc.)
*   Limón (para el pescado)



# Parte 2 - Demo para generar menús y listas en base a las preferencias del usuario

1. Crea un dropdown de tipo de dieta: "Vegetariana", "Vegana", "Sin gluten", "Mediterránea", "Cetogénica (Keto)"...

2. Usa la componente gr.Markdown() para mostrar los outputs de forma más bonita. Pide explícitamente al modelo que la respuesta esté formateada en Markdown.

In [7]:
import gradio as gr


menu_user_prompt_template = "Genera un menú semanal basado en una dieta {preferencia} con comida y cena. Solo devuelve la tabla de menú en Markdown."

lista_user_prompt_template = "Genera la lista de la compra con emojis para este menú {menu_generado}. Devuelve solo la lista en Markdown."

# Función principal que llama a ambas funciones
def genera_menu_y_lista(preferencia):
    menu = generate_text(menu_user_prompt_template.format(preferencia=preferencia))
    lista = generate_text(lista_user_prompt_template.format(menu_generado=menu))
    return menu, lista

# Definir inputs y outputs
inputs = [
    gr.Dropdown(
        label="Tipo de dieta",
        choices=[
            "Vegetariana",
            "Vegana",
            "Sin gluten",
            "Mediterránea",
            "Cetogénica (Keto)",
        ],
        value="Vegetariana"
    )
]

outputs = [
    gr.Markdown(label="# Menú Semanal"),
    gr.Markdown(label="# Lista de Compras")
]

# Ejemplos
examples = [
    ["Vegana"],
    ["Cetogénica (Keto)"],
    ["Dieta baja en grasas"],
]

# Interfaz con Gradio
demo = gr.Interface(
    fn=genera_menu_y_lista,
    inputs=inputs,
    outputs=outputs,
    title="Generador de Menú Semanal con Gemini",
    description="Selecciona una preferencia dietética para obtener un menú semanal y la lista de compras basada en el menú generado.",
    examples=examples
)

demo.launch()

/usr/local/lib/python3.11/dist-packages/gradio/components/dropdown.py:226: UserWarning: The value passed into gr.Dropdown() is not in the list of choices. Please update the list of choices to include: Dieta baja en grasas or set allow_custom_value=True.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43fc1169332614eddc.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
